# Monte Carlo with rigid cluster moves


In this example, a low density and low temperature simulation of Lennard-Jones particles is conducted with and without rigid cluster moves and with and without single particle translations.
A droplet is expected to form at these conditions.
Without cluster moves, this droplet is unable to translate and rotate freely once it is formed.
Single particle translations are still important.
In order to obey detailed balance with rigid cluster moves, clusters cannot be created or destroyed.
Thus, single particle translations are still required to form a droplet.
Without single particle translations, particle randomly dispersed will not be allowed to coalesce.

The three possible combinations are listed as follows:
- sim 1 - single particle translations only
- sim 2 - rigid cluster moves only
- sim 3 - both

In [1]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/monte_carlo/tutorial/')
import analyze

for sim in [1, 2, 3]: # see combinations described above
    monte_carlo = feasst.MonteCarlo()
    monte_carlo.add(feasst.Configuration(feasst.args(
        {"cubic_box_length": "8",
         "particle_type": feasst.install_dir() + "/forcefield/data.lj"})))
    if sim == 1:
        monte_carlo.add(feasst.Potential(feasst.MakeLennardJones()))
    else:
        monte_carlo.add(feasst.Potential(feasst.MakeLennardJones(),
                        feasst.MakeVisitModel(feasst.MakeVisitModelInner(feasst.MakeEnergyMapAll()))));
    monte_carlo.add(feasst.MakeMetropolis(feasst.args(
        {"beta": "4", "chemical_potential": "1."})))
    if sim == 1 or sim == 3:
        monte_carlo.add(feasst.MakeTrialTranslate(feasst.args(
            {"tunable_param": "2.", "tunable_target_acceptance": "0.2"})))
    monte_carlo.seek_num_particles(25)
    
    # record the size of the cluster of the first particle for comparison
    if sim == 2 or sim == 3:
        cluster_criteria = feasst.MakeClusterCriteria(feasst.args({'energy_maximum': '-0.5'}))
        select_cluster = feasst.TrialSelectCluster(cluster_criteria)
        select_cluster.select_cluster(0, monte_carlo.system())
        cluster_size = select_cluster.mobile().num_particles()
        feasst.add_rigid_cluster_trials(monte_carlo, cluster_criteria)
    steps_per = int(1e3)
    monte_carlo.add(feasst.MakeTuner(feasst.args({"steps_per" : str(steps_per)})))
    monte_carlo.add(feasst.MakeLog(feasst.args({"steps_per" : str(steps_per)})))
    monte_carlo.add(feasst.MakeCPUTime(feasst.args({"steps_per" : str(steps_per),
                                                    'file_name': 'cpu'+str(sim)+'.txt'})))
    moviename='movie'+str(sim)+'.xyz'
    monte_carlo.add(feasst.MakeMovie(feasst.args(
        {"steps_per" : str(steps_per), "file_name" : moviename, "clear_file": "True"})))
    monte_carlo.add(feasst.MakeCheckEnergy(feasst.args(
        {"steps_per" : str(steps_per), "tolerance" : "1e-8"})))
    monte_carlo.attempt(int(1e5))
    
    # With rigid cluster moves only, clusters cannot coalesce or break up.
    # So the clusters in the original configuration remain.
    if sim == 2 or sim == 3:
        select_cluster.select_cluster(0, monte_carlo.system())
        if sim == 2:
            assert(select_cluster.mobile().num_particles() == cluster_size)
        # otherwise, we expect the formation of a droplet
        if sim == 3:
            cluster_size_av = monte_carlo.trial(1).stage(0).trial_select().printable("cluster_size").average()
            assert(cluster_size_av > 20)

Take a look at the cpu files. Sim 1 is faster than sim 2 because cluster moves are more expensive. Even when clusters are small, energy map book keeping and cluster calculations can be slow compared to a simple LJ potential.

Sim 3 is even slower than sim 2 because sim 3 forms larger clusters. Each step moves many particles and is more expensive.

Note that, even if sim 3 takes the longest time to complete a given number of steps, it can still be considered the most efficient sim due to improved sampling. Essentially, sims 1 and 2 are non ergodic and get trapped in a particular region of phase space, while sim 3 can sample all of phase space.